# Feature Engineering

#### Import the required libraries

In [1]:
# NumPy for numerical computing
import numpy as np

# Pandas for DataFrames
import pandas as pd
pd.set_option('display.max_columns', 100)

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline 

# Seaborn for visualization
import seaborn as sns

#### Import the cleaned dataset

In [15]:
# Load cleaned dataset from the previous lecture
df = pd.read_csv('cleaned_df.csv')
df.head(5)

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,property_type
0,295850,2013,234,81,1,1,584,2013,0,0.0,Condo
1,216500,2006,169,51,1,1,612,1965,0,1.0,Condo
2,279900,2012,216,74,1,1,615,1963,0,0.0,Condo
3,379900,2005,265,92,1,1,618,2000,33541,0.0,Condo
4,340000,2002,88,30,1,1,634,1992,0,0.0,Condo


## I. Domain Knowledge

#### A. Popular Properties

2 bedroom and 2 bathroom properties are especially popular for investors. Let's create an indicator variable just for properties with 2 beds and 2 baths.

In [3]:
# Build your code step by step
#((df.beds == 2) & (df.baths == 2))
# ((df.beds == 2) & (df.baths == 2)).astype(int)

In [16]:
# Create indicator variable for properties with 2 beds and 2 baths
df['popular'] = ((df.beds == 2) & (df.baths == 2)).astype(int)

In [18]:
# Check how many propoerties have 2 baths and 2 beds 
df.popular.value_counts()

0    1704
1     178
Name: popular, dtype: int64

#### B. Housing Market Recession

We are modeling housing prices in the United States, it's important to consider the housing market recession around 2008. According to data from Zillow, the lowest housing prices were from 2010 to end of 2013.

<br>
Create an indicator feature **recession**

Here's how:
* Your first condition `year_sold >= 2010`
* Your second condition `year_sold <= 2013`
* Combine the two conditions with an `&` operator
* Convert the resulting data to `int` type.

In [19]:
# Create a new variable recession
df['recession']=((df.year_sold >= 2010) & (df.year_sold <= 2013)).astype(int)

In [20]:
# Check how many propoerties were sold during recession period 
df.recession.value_counts()

0    1386
1     496
Name: recession, dtype: int64

In [21]:
fd = df[['year_sold','recession']]
fd.head(8)


,year_sold,recession
0,2013,1
1,2006,0
2,2012,1
3,2005,0
4,2002,0
5,2004,0
6,2011,1
7,2005,0


## II. Interaction Features

In the first step, you engineered features from domain knowledge. interaction features can be products, sums, or differences between two features.

#### A. Property Age

We have the features `year_sold` and the `year_built`. let's create a new feature `property_age`

In [8]:
df.property_age = df.year_sold - df.year_built 

C:\Users\irish\AppData\Local\Temp\ipykernel_8236\2248682832.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.property_age = df.year_sold - df.year_built


In [9]:
type(df.property_age)

pandas.core.series.Series

In [22]:
# Create a 'property_age' feature
df['property_age']=(df.year_sold - df.year_built ).astype(int)

Do a quick sanity check on that feature. Run `df.describe()` and check the stats for the feature `property_age`

In [23]:
# Do you see any error?
df.describe()

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,popular,recession,property_age
count,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000
mean,422908.798618,2007.107864,466.903294,140.486716,3.421360,2.580765,2330.171626,1982.981403,12751.197131,0.880446,0.094580,0.263549,24.126461
std,151473.251553,5.195851,231.653858,72.935379,1.068335,0.945125,1336.926475,20.287099,35304.268897,0.324525,0.292712,0.440675,21.153271
min,200000.000000,1993.000000,88.000000,30.000000,1.000000,1.000000,500.000000,1880.000000,0.000000,0.000000,0.000000,0.000000,-8.000000
25%,300000.000000,2004.000000,320.000000,94.000000,3.000000,2.000000,1346.000000,1970.000000,1542.000000,1.000000,0.000000,0.000000,6.000000
50%,392000.000000,2007.000000,426.000000,125.000000,3.000000,3.000000,1907.500000,1986.000000,6074.000000,1.000000,0.000000,0.000000,20.000000
75%,525000.000000,2011.000000,569.000000,169.000000,4.000000,3.000000,3005.000000,2000.000000,11761.000000,1.000000,0.000000,1.000000,38.000000
max,800000.000000,2016.000000,4508.000000,1374.000000,5.000000,6.000000,8450.000000,2015.000000,436471.000000,1.000000,1.000000,1.000000,114.000000


In [24]:
# Check number of observations with 'property_age' < 0
(df.property_age < 0).sum()

19

On second thought, this could be an error or that some homeowners buy houses before the construction company builts them. But for the purpose of this project we will remove these observations.

We'll do a quick ad-hoc data cleaning and remove these observations from our dataset.

#### Remove observations where `property_age` is less than 0.
* Keep only observations where `property_age` is 0 and above.

In [25]:
# Print df shape before
print(df.shape)

# Remove rows where property_age is less than 0
df = df[df.property_age >= 0]

# Print number of rows in remaining dataframe
print(df.shape)

(1882, 14)
(1863, 14)


### III. Drop Redundant Features

Beacuse we created a new feature `property_age` using features `year_built` and `year_sold`. We can drop those two features.

**Remove features 'year_built' and 'year_sold'** 
* Use Pandas's `.drop()` function.
* Remember to set `axis=1` because you are dropping columns.
* Remember to do 'inplace=True'.


In [26]:
# Drop 'year_built' and 'year_sold' from the dataset
df.drop(['year_built', 'year_sold'], axis=1, inplace=True)
df.head()
df.corr()

C:\Users\irish\AppData\Local\Temp\ipykernel_8236\4144439370.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,price,property_tax,insurance,beds,baths,sqft,lot_size,basement,popular,recession,property_age
price,1.000000,0.558266,0.554583,0.387682,0.398615,0.486858,0.185244,0.109144,-0.148108,-0.022207,-0.005428
property_tax,0.558266,1.000000,0.983430,0.544898,0.507861,0.640393,0.218076,0.182701,-0.208959,-0.121395,0.012119
insurance,0.554583,0.983430,1.000000,0.479399,0.454178,0.569497,0.188731,0.135135,-0.173812,-0.120809,0.075054
beds,0.387682,0.544898,0.479399,1.000000,0.640168,0.688634,0.228757,0.413082,-0.430083,-0.070744,-0.015989
baths,0.398615,0.507861,0.454178,0.640168,1.000000,0.669070,0.186494,0.344052,-0.197480,-0.057730,-0.213489
sqft,0.486858,0.640393,0.569497,0.688634,0.669070,1.000000,0.325832,0.270175,-0.270799,-0.078903,-0.284702
lot_size,0.185244,0.218076,0.188731,0.228757,0.186494,0.325832,1.000000,0.061934,-0.092659,0.006077,-0.042227
basement,0.109144,0.182701,0.135135,0.413082,0.344052,0.270175,0.061934,1.000000,-0.198936,-0.047486,-0.018488
popular,-0.148108,-0.208959,-0.173812,-0.430083,-0.197480,-0.270799,-0.092659,-0.198936,1.000000,0.083255,-0.083813
recession,-0.022207,-0.121395,-0.120809,-0.070744,-0.057730,-0.078903,0.006077,-0.047486,0.083255,1.000000,0.097146


### Save the final dataset

We will save this dataset and train our model on it.

In [28]:
# Save the data as 'final.csv'
df.to_csv('final.csv', index=None)